In [1]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import pickle

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

In [3]:
def onehot(vals, possible_vals):
    if not isinstance(possible_vals, list): raise TypeError("provide possible_vals as a list")
    enc_vals = np.zeros([len(vals), len(possible_vals)])
    for i, value in enumerate(vals):
        if isinstance(possible_vals[0], float):
            enc = np.where(abs(possible_vals-value)<1e-3)
        else:
            enc = np.where(possible_vals==value)
        enc_vals[i,enc] = 1
    return enc_vals

In [4]:
class Rescale(object):
    def __init__(self, scalar):
        self.scalar = scalar

    def __call__(self, im):
        w, h = [int(s*self.scalar) for s in im.size]
        return transforms.Resize((h, w))(im)

class Crop(object):
    def __init__(self, box):
        assert len(box) == 4
        self.box = box

    def __call__(self, im):
        return im.crop(self.box)

class Augment(object):
    def __init__(self, seq):
        self.seq = seq

    def __call__(self, im):
        return Image.fromarray(self.seq.augment_images([np.array(im)])[0])

def get_data_transforms(t='train'):
    data_transforms = {
        'train': transforms.Compose([Crop((0,120,800,480)),
            Rescale(0.4),
            transforms.ToTensor()
        ]),
        'val': transforms.Compose([
            Crop((0,120,800,480)),
            Rescale(0.4),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }
    return data_transforms[t]

In [59]:
from torch.utils.data import Dataset
from matplotlib import image
from matplotlib import pyplot
from torchvision import transforms
from PIL import Image

#LABEL_KEYS = ['v_throttle', 'v_break', 'v_steer', 'traffic_light', 'front_vehicle', 'centre_dist', 'pedestrian_distance']

class CARLA_Dataset(Dataset):
    def __init__(self, t, pickle_file_path_l, image_dir_path_l, pickle_file_path_r, image_dir_path_r):
        self.inputs, self.labels = {}, {}
        #initializing transforms
        assert t in ['train', 'val']
        self.transform = get_data_transforms(t)
        
        ###############LEFT CAMERA##############
        #reading the title of all images to access the pickle file
        self.image_path_l = image_dir_path_l
        self.image_list_l = os.listdir(image_dir_path_l)
        self.image_list_l.remove('.DS_Store')
        self.file_name_l = []
        for file in self.image_list_l:
            self.file_name_l.append(os.path.splitext(file)[0])
            
        #initialize the labels which should be returned
        self.labels_l = {}
        
        #read pickle file
        self.pickle_list_l = os.listdir(pickle_file_path_l)
        self.pickled_data_l = {}
        for file in self.pickle_list_l:
            f = open((pickle_file_path_l + file), 'rb')  
            self.pickled_data_l.update(pickle.load(f))
            f.close() 
        
        ###############RIGHT CAMERA##############
        #reading the title of all images to access the pickle file
        self.image_path_r = image_dir_path_r
        self.image_list_r = os.listdir(image_dir_path_r)
        self.image_list_r.remove('.DS_Store')
        self.file_name_r = []
        for file in self.image_list_r:
            #self.file_name.append(file[0:16])
            self.file_name_r.append(os.path.splitext(file)[0])
            
        #initialize the labels which should be returned
        self.labels_r = {}
        
        #read pickle file
        self.pickle_list_r = os.listdir(pickle_file_path_r)
        self.pickled_data_r = {}
        for file in self.pickle_list_r:
            f = open((pickle_file_path_r + file), 'rb')  
            self.pickled_data_r.update(pickle.load(f))
            f.close() 
        
        # transform reg affordances
        #reg_keys = ['front_vehicle', 'centre_dist', 'pedestrian_distance']
        #for key in self.pickled_data:
        #   entry = self.pickled_data[key]
            #print("1.", entry['front_vehicle'])
        #reg_norm = entry[reg_keys] / abs(entry[reg_keys]).max()
                # transform cls affordances
      
        f.close() 

    def __len__(self):
        return len(self.file_name_l)

    def __getitem__(self, idx):
        frames      = []
        #####################LEFT CAMERA################
        #reading PIL
        self.raw_image_l = Image.open(os.path.join(self.image_path_l, self.image_list_l[idx]))
        #pyplot.imshow(self.raw_image_l)
        #pyplot.show()
        
        #reading file name to access the pickle file
        current_fname_l = self.file_name_l[idx]
        label_dict_l = self.pickled_data_l[current_fname_l]       
        label_values_l = list(label_dict_l.values())
        
        #transforming regression labels
        self.labels_l['front_vehicle'] = torch.Tensor(np.array(float(label_dict_l['front_vehicle'])))
        self.labels_l['centre_dist'] = torch.Tensor(np.array(float(label_dict_l['centre_dist'])))
        self.labels_l['pedestrian_distance'] = torch.Tensor(np.array(float(label_dict_l['pedestrian_distance'])))
        
        #transforming classification labels
        self.labels_l['traffic_light'] = torch.Tensor(onehot(np.array(label_dict_l['traffic_light']), [False, 'Green', True, 'Red']))
        #transforming raw PIL image
        im_l = self.transform(self.raw_image_l)
        
        ###############RIGHT CAMERA####################
        #reading PIL
        self.raw_image_r = Image.open(os.path.join(self.image_path_r, self.image_list_r[idx]))
        #pyplot.imshow(self.raw_image_r)
        #pyplot.show()
        
        #reading file name to access the pickle file
        current_fname_r = self.file_name_r[idx]
        label_dict_r = self.pickled_data_r[current_fname_r]       
        label_values_r = list(label_dict_r.values())
       
        #transforming regression labels
        self.labels_r['front_vehicle'] = torch.Tensor(np.array(float(label_dict_r['front_vehicle'])))
        self.labels_r['centre_dist'] = torch.Tensor(np.array(float(label_dict_r['centre_dist'])))
        self.labels_r['pedestrian_distance'] = torch.Tensor(np.array(float(label_dict_r['pedestrian_distance'])))
        
        #transforming classification labels
        self.labels_r['traffic_light'] = torch.Tensor(onehot(np.array(label_dict_r['traffic_light']), [False, 'Green', True, 'Red']))
        
        #transforming raw PIL image
        im_r = self.transform(self.raw_image_r)
        
        #return transformed image and labels
        sample = {'image_l' : im_l, 'labels_l' : self.labels_l ,'image_r' : im_r, 'labels_r' : self.labels_r}
        return sample
        
        #self.labels['v_break'] = torch.Tensor(onehot(np.array(list(label_dict['v_break'])), [0.0, 1.0]))
        #print(label_dict['traffic_light'])
        #print(self.labels['traffic_light'])
        #print(label_dict['v_break'])
        #print(self.labels['v_break'])
        #pyplot.imshow(self.raw_image)
        #pyplot.show()
        #reg_keys = ('front_vehicle', 'centre_dist', 'pedestrian_distance', 'v_steer')
        #reg_norm = label_dict[reg_keys] / abs(label_dict[reg_keys]).max()
        #reg_norm = label_dict[reg_keys]

        #for i in 'front_vehicle', 'centre_dist', 'pedestrian_distance', 'v_steer':
        #  reg_norm = label_dict[i] / abs(label_dict[i])
        #  print(i, reg_norm)
        #print(type(reg_keys))
        #reg_norm = label_dict['front_vehicle', 'centre_dist'] / max(abs(label_dict['front_vehicle', 'centre_dist']))

In [63]:
#C_dataset = CARLA_Dataset( 'train', pickle_file_path_l='/Users/seimasaki/CE7454_2019/codes/data/Left_pickle_file/', image_dir_path_l='/Users/seimasaki/CE7454_2019/codes/data/Left_camera/', pickle_file_path_r='/Users/seimasaki/CE7454_2019/codes/data/Right_pickle_file/', image_dir_path_r='/Users/seimasaki/CE7454_2019/codes/data/Right_camera/', )

#for i in range(len(C_dataset)):
 # sample = C_dataset[i]
  #print(sample['image_l'])
 # print(sample['labels_l'])

In [64]:
def get_data(batch_size):
    train_ds = CARLA_Dataset( 'train', pickle_file_path_l='/Users/seimasaki/CE7454_2019/codes/data/Left_pickle_file/',
                                    image_dir_path_l='/Users/seimasaki/CE7454_2019/codes/data/Left_camera/', 
                                    pickle_file_path_r='/Users/seimasaki/CE7454_2019/codes/data/Right_pickle_file/',
                                    image_dir_path_r='/Users/seimasaki/CE7454_2019/codes/data/Right_camera/', )
    val_ds = CARLA_Dataset( 'val', pickle_file_path_l='/Users/seimasaki/CE7454_2019/codes/data/Left_pickle_file/',
                                    image_dir_path_l='/Users/seimasaki/CE7454_2019/codes/data/Left_camera/', 
                                    pickle_file_path_r='/Users/seimasaki/CE7454_2019/codes/data/Right_pickle_file/',
                                    image_dir_path_r='/Users/seimasaki/CE7454_2019/codes/data/Right_camera/', )
    return (
        DataLoader(train_ds, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=10),
        #DataLoader(val_ds, batch_size=batch_size*2, pin_memory=True, num_workers=10),
    )

In [65]:
print(get_data(100))

(<torch.utils.data.dataloader.DataLoader object at 0x125bb4128>, <torch.utils.data.dataloader.DataLoader object at 0x124dc8518>)
